In [1]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable

In [2]:
purity = 0.5 + 0.5*np.random.rand()
target_vector = np.random.normal(0, 1, 3)
target_vector = target_vector/(np.sum(target_vector**2))
target_vector = target_vector*(2*purity - 1)**0.5
target_vector = Variable(torch.tensor(target_vector ))

print(target_vector)    
print(purity)


tensor([ 0.1843,  0.2918, -0.2909], dtype=torch.float64)
0.779881733078468


In [3]:
device = qml.device('qiskit.aer', wires=3, backend='qasm_simulator')
params = np.random.normal(0,np.pi, (3, 2, 3))
params = Variable(torch.tensor(params), requires_grad=True)

@qml.qnode(device, interface="torch")
def circuit(params, M=None):
    j=0
    for i in range(3):
        qml.RX(params[i, j, 0], wires=i)
        qml.RY(params[i, j, 1], wires=i)
        qml.RZ(params[i, j, 2], wires=i)
        
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[1, 2])

    j=1
    for i in range(3):
        qml.RX(params[i, j, 0], wires=i)
        qml.RY(params[i, j, 1], wires=i)
        qml.RZ(params[i, j, 2], wires=i)
        
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[1, 2])
    
    return qml.expval(qml.Hermitian(M, wires=0))


In [9]:
Paulis = Variable(torch.zeros([3, 2, 2], dtype=torch.complex128), requires_grad=False)
Paulis[0] = torch.tensor([[0, 1], [1, 0]])
Paulis[1] = torch.tensor([[0, -1j], [1j, 0]])
Paulis[2] = torch.tensor([[1, 0], [0, -1]])

def cost(params):
    L = 0
    for k in range(3):
        L += torch.abs(circuit(params, Paulis[k]) - target_vector[k])
    return L

opt = torch.optim.Adam([params], lr=0.1)
best_loss = 1*cost(params)
best_params = 1*params

In [10]:
for epoch in range(32):
    opt.zero_grad()
    loss = cost(params)
    print(epoch, loss)
    loss.backward()
    opt.step()

    if loss < best_loss:
        best_loss = 1*loss
        best_params = 1*params

0 tensor(2.0031, dtype=torch.float64, grad_fn=<AddBackward0>)
1 tensor(1.3117, dtype=torch.float64, grad_fn=<AddBackward0>)
2 tensor(0.8039, dtype=torch.float64, grad_fn=<AddBackward0>)
3 tensor(0.7223, dtype=torch.float64, grad_fn=<AddBackward0>)
4 tensor(0.5446, dtype=torch.float64, grad_fn=<AddBackward0>)
5 tensor(0.5148, dtype=torch.float64, grad_fn=<AddBackward0>)
6 tensor(0.5110, dtype=torch.float64, grad_fn=<AddBackward0>)
7 tensor(0.5735, dtype=torch.float64, grad_fn=<AddBackward0>)
8 tensor(0.3898, dtype=torch.float64, grad_fn=<AddBackward0>)
9 tensor(0.3449, dtype=torch.float64, grad_fn=<AddBackward0>)
10 tensor(0.3019, dtype=torch.float64, grad_fn=<AddBackward0>)
11 tensor(0.2239, dtype=torch.float64, grad_fn=<AddBackward0>)
12 tensor(0.0758, dtype=torch.float64, grad_fn=<AddBackward0>)
13 tensor(0.0133, dtype=torch.float64, grad_fn=<AddBackward0>)
14 tensor(0.2367, dtype=torch.float64, grad_fn=<AddBackward0>)
15 tensor(0.2523, dtype=torch.float64, grad_fn=<AddBackward0>)
16

In [11]:
X_exp = circuit(params, Paulis[0]).item()
Y_exp = circuit(params, Paulis[1]).item()
Z_exp = circuit(params, Paulis[2]).item()
print(X_exp, Y_exp, Z_exp)

-0.48828125 -0.478515625 0.568359375


In [12]:
print(target_vector.numpy())

[-0.51246847 -0.41497495  0.5463451 ]


In [13]:
rho = 0.5*(torch.eye(2) + X_exp*Paulis[0] + Y_exp*Paulis[1] + Z_exp*Paulis[2]).detach().numpy()
print(rho)

[[ 0.78417969+0.j         -0.24414062+0.23925781j]
 [-0.24414062-0.23925781j  0.21582031+0.j        ]]
